![Credit card being held in hand](credit_card.jpg)

Commercial banks receive _a lot_ of applications for credit cards. Many of them get rejected for many reasons, like high loan balances, low income levels, or too many inquiries on an individual's credit report, for example. Manually analyzing these applications is mundane, error-prone, and time-consuming (and time is money!). Luckily, this task can be automated with the power of machine learning and pretty much every commercial bank does so nowadays. In this workbook, you will build an automatic credit card approval predictor using machine learning techniques, just like real banks do.

### The Data

The data is a small subset of the Credit Card Approval dataset from the UCI Machine Learning Repository showing the credit card applications a bank receives. This dataset has been loaded as a `pandas` DataFrame called `cc_apps`. The last column in the dataset is the target value.

In [72]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

# Load the dataset
cc_apps = pd.read_csv("cc_approvals.data", header=None) 
cc_apps.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,g,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,g,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,g,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,g,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,s,0,+


In [73]:
cc_apps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       690 non-null    object 
 1   1       690 non-null    object 
 2   2       690 non-null    float64
 3   3       690 non-null    object 
 4   4       690 non-null    object 
 5   5       690 non-null    object 
 6   6       690 non-null    object 
 7   7       690 non-null    float64
 8   8       690 non-null    object 
 9   9       690 non-null    object 
 10  10      690 non-null    int64  
 11  11      690 non-null    object 
 12  12      690 non-null    int64  
 13  13      690 non-null    object 
dtypes: float64(2), int64(2), object(10)
memory usage: 75.6+ KB


In [74]:
cc_apps.iloc[:,13].unique()

array(['+', '-'], dtype=object)

In [75]:
# missing values 
print(cc_apps.head())
print(cc_apps.describe())

# for column in cc_apps.columns:
#    print(column, cc_apps[column].value_counts())

# there seems to be missing values in the columns from above analysis

# fill the missing values with np.nan
import numpy as np

cc_apps = cc_apps.replace('?', np.nan)

# for column in cc_apps.columns:
#    print(column, cc_apps[column].value_counts())
cc_apps.tail(100)

  0      1      2  3  4  5  6     7  8  9   10 11   12 13
0  b  30.83  0.000  u  g  w  v  1.25  t  t   1  g    0  +
1  a  58.67  4.460  u  g  q  h  3.04  t  t   6  g  560  +
2  a  24.50  0.500  u  g  q  h  1.50  t  f   0  g  824  +
3  b  27.83  1.540  u  g  w  v  3.75  t  t   5  g    3  +
4  b  20.17  5.625  u  g  w  v  1.71  t  f   0  s    0  +
               2           7          10             12
count  690.000000  690.000000  690.00000     690.000000
mean     4.758725    2.223406    2.40000    1017.385507
std      4.978163    3.346513    4.86294    5210.102598
min      0.000000    0.000000    0.00000       0.000000
25%      1.000000    0.165000    0.00000       0.000000
50%      2.750000    1.000000    0.00000       5.000000
75%      7.207500    2.625000    3.00000     395.500000
max     28.000000   28.500000   67.00000  100000.000000


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
590,b,30.17,6.500,u,g,cc,v,3.125,t,t,8,g,1200,+
591,b,27.00,0.750,u,g,c,h,4.250,t,t,3,g,150,+
592,b,23.17,0.000,NaN,NaN,NaN,NaN,0.000,f,f,0,p,0,+
593,b,34.17,5.250,u,g,w,v,0.085,f,f,0,g,6,+
594,b,38.67,0.210,u,g,k,v,0.085,t,f,0,g,0,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,b,21.08,10.085,y,p,e,h,1.250,f,f,0,g,0,-
686,a,22.67,0.750,u,g,c,v,2.000,f,t,2,g,394,-
687,a,25.25,13.500,y,p,ff,ff,2.000,f,t,1,g,1,-
688,b,17.92,0.205,u,g,aa,v,0.040,f,f,0,g,750,-


In [76]:
# preprocessing steps
'''
1. fill the numeric columns with np.nan with mean?
2. encode the non numeric columns to numeric after using mode fill
3. apply standard scaling of the columns before evaluation
'''
# 1 & 2
from sklearn.preprocessing import LabelEncoder

non_numeric_cols = []
model = LabelEncoder()
for column in cc_apps.columns:
    if cc_apps[column].dtypes == 'object':
        non_numeric_cols.append(column)
        cc_apps = cc_apps.fillna(cc_apps[column].value_counts().index[0])
        # cc_apps[column]= model.fit_transform(cc_apps[column])
    else:
        cc_apps = cc_apps.fillna(cc_apps[column].mean())
# print(cc_apps.tail(100))
# print(non_numeric_cols)
print(cc_apps.describe())


for column in cc_apps.columns:
    if cc_apps[column].dtypes == 'object':
        cc_apps[column]= model.fit_transform(cc_apps[column])
# print(cc_apps.describe())

# 3
X = cc_apps[[0, 1, 2,3, 4, 5, 6, 7, 8, 9, 10, 11, 12]]
y = cc_apps[[13]]
X, y

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=21)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

               2           7          10             12
count  690.000000  690.000000  690.00000     690.000000
mean     4.758725    2.223406    2.40000    1017.385507
std      4.978163    3.346513    4.86294    5210.102598
min      0.000000    0.000000    0.00000       0.000000
25%      1.000000    0.165000    0.00000       0.000000
50%      2.750000    1.000000    0.00000       5.000000
75%      7.207500    2.625000    3.00000     395.500000
max     28.000000   28.500000   67.00000  100000.000000


In [77]:
# ml model

logreg = LogisticRegression()
logreg.fit(X_train_scaled, y_train)

from sklearn.metrics import confusion_matrix
# prediction
y_pred = logreg.predict(X_test_scaled)

print("Accuracy of logistic regression classifier: ", logreg.score(X_test_scaled, y_test))

confusion_matrix(y_pred, y_test)

Accuracy of logistic regression classifier:  0.838150289017341


array([[70, 19],
       [ 9, 75]])

In [78]:
# ml models - checking if better accuracy can be achieved with CV

from sklearn.model_selection import GridSearchCV

params = {'tol': np.linspace(0.001, 1.0, 25), 'max_iter':[250 for i in range(25)]}

grid_model = GridSearchCV(estimator=logreg, param_grid=params, cv=5)
X_scaled = scaler.fit_transform(X)

grid_model_result = grid_model.fit(X_scaled, y)

# Summarize results
best_score, best_params = grid_model_result.best_score_, grid_model_result.best_params_
print("Best accuracy so far: %f using %s" % (best_score, best_params))

Best accuracy so far: 0.853623 using {'max_iter': 250, 'tol': 0.9167500000000001}


In [79]:
# highest achived accuracy on aboev methods is around 85%